# Evaluate trained models
In this notebook learned models, which are managed by *mlflow* are evaluated with respect to different metrics. 

In [5]:
#!pip install mlflow

In [32]:
import mlflow
import mlflow.keras
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
#!pip install --upgrade tensorflow

In [8]:
#!conda install -y -c conda-forge tensorflow=2.5.0

In [9]:
import tensorflow as tf

In [33]:
tf.__version__

'2.7.0'

In [34]:
mlflow.__version__

'1.24.0'

## Display all runs within the selected mlflow-experiment-ID
Select experiment:

In [54]:
mlflow.get_experiment_by_name("Reuters Newswire Topic Classification")

<Experiment: artifact_location='file:///Users/johannes/gitprojects/dsmmlbook/mlbook/mlflowExperiments/mlruns/2', experiment_id='2', lifecycle_stage='active', name='Reuters Newswire Topic Classification', tags={}>

In [55]:
mlflow.set_experiment("Reuters Newswire Topic Classification")

<Experiment: artifact_location='file:///Users/johannes/gitprojects/dsmmlbook/mlbook/mlflowExperiments/mlruns/2', experiment_id='2', lifecycle_stage='active', name='Reuters Newswire Topic Classification', tags={}>

In [56]:
allruns=mlflow.search_runs()
allruns.columns

Traceback (most recent call last):
  File "/Users/johannes/opt/anaconda3/envs/ss22/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 732, in _list_run_infos
    run_info = self._get_run_info_from_dir(r_dir)
  File "/Users/johannes/opt/anaconda3/envs/ss22/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 562, in _get_run_info_from_dir
    meta = read_yaml(run_dir, FileStore.META_DATA_FILE_NAME)
  File "/Users/johannes/opt/anaconda3/envs/ss22/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 176, in read_yaml
    raise MissingConfigException("Yaml file '%s' does not exist." % file_path)
mlflow.exceptions.MissingConfigException: Yaml file '/Users/johannes/gitprojects/dsmmlbook/mlbook/mlflowExperiments/mlruns/2/.ipynb_checkpoints/meta.yaml' does not exist.


Index(['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time',
       'end_time', 'metrics.val_loss', 'metrics.accuracy', 'metrics.loss',
       'metrics.val_accuracy', 'params.initial_epoch', 'params.learning_rate',
       'params.epochs', 'params.validation_steps', 'params.optimizer_name',
       'params.shuffle', 'params.max_queue_size', 'params.use_multiprocessing',
       'params.steps_per_epoch', 'params.num_layers', 'params.epsilon',
       'params.workers', 'params.validation_split', 'params.validation_freq',
       'params.class_weight', 'params.batch_size', 'params.sample_weight',
       'params.validation_batch_size', 'tags.mlflow.user',
       'tags.mlflow.source.name', 'tags.mlflow.log-model.history',
       'tags.mlflow.source.type', 'tags.mlflow.autologging'],
      dtype='object')

In [57]:
allruns

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.val_loss,metrics.accuracy,metrics.loss,metrics.val_accuracy,...,params.validation_freq,params.class_weight,params.batch_size,params.sample_weight,params.validation_batch_size,tags.mlflow.user,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.source.type,tags.mlflow.autologging
0,785db4c9290e49858d9aa7862c13b95a,2,FINISHED,file:///Users/johannes/gitprojects/dsmmlbook/m...,2022-04-04 15:21:32.625000+00:00,2022-04-04 15:21:41.702000+00:00,0.947013,0.944451,0.219746,0.791991,...,1,None,32,None,None,johannes,/Users/johannes/opt/anaconda3/envs/ss22/lib/py...,"[{""run_id"": ""785db4c9290e49858d9aa7862c13b95a""...",LOCAL,keras


## Select model, that shall be evaluated

Select model ID:

In [50]:
ID=0

Get run id:

In [51]:
id=allruns.loc[ID,"run_id"]
id

'785db4c9290e49858d9aa7862c13b95a'

Delete a run:

In [41]:
#mlflow.tracking.MlflowClient().delete_run(id)

Delete a experiment:

In [20]:
#mlflow.tracking.MlflowClient().delete_experiment("2")

Determine location of selected model:

In [21]:
locationBase=allruns.loc[ID,"artifact_uri"]
locationBase

'file:///Users/johannes/gitprojects/dsmmlbook/mlbook/mlflowExperiments/mlruns/1/b6ec669b90984731a09ca8ead7e65222/artifacts'

In [22]:
start=locationBase.index("mlruns")
print(start)
locationBaseRelativePath="."+locationBase[int(start-1):]
print(locationBaseRelativePath)

70
./mlruns/1/b6ec669b90984731a09ca8ead7e65222/artifacts


In [23]:
location=locationBaseRelativePath+"/model/"

## Load selected trained model and evaluate it

In [27]:
mlp_model = mlflow.keras.load_model(location)

In [29]:
mlp_model

Ridge(alpha=0.5)

In [30]:
from tensorflow.keras.datasets import reuters

In [ ]:
max_words = 1000
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,
                                                         test_split=0.2)

In [38]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_words)
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
val_pred=mlp_model.predict(x_test)

In [39]:
from sklearn.metrics import confusion_matrix

In [40]:
num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

46 classes


In [41]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [42]:
y_test.shape

(2246, 46)

In [43]:
confmat=confusion_matrix(y_test.argmax(axis=1),val_pred.argmax(axis=1))

In [44]:
pd.DataFrame(confmat)

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,8,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,80,0,5,7,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,11,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,3,0,765,33,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,1,5,0,36,410,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0
5,0,4,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,2,0,11,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,5,2,0,0,0,28,0,...,0,0,0,0,0,1,0,0,0,0
9,0,1,0,0,3,0,0,0,1,17,...,0,0,0,0,0,0,0,0,0,0


In [45]:
score = mlp_model.evaluate(x_test, y_test,
                       batch_size=32, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

71/71 [==============================] - 0s 1ms/step - loss: 0.9956 - accuracy: 0.7769
Test score: 0.9955558180809021
Test accuracy: 0.7769367694854736
